# Using Thymio with Python

Thymio robots may also be controlled using python using Jupyter notebooks. For that you need to install tdmclient.

References

- https://pypi.org/project/tdmclient/
- https://drive.google.com/file/d/1ePsAWWnFkwqS168_Ae1xSM8vGioWO6cJ/view





In [ ]:
#import os
#os.system("pip install tdmclient")

Next, start Thymio Suite and connect to a robot (real or simulated).

Load the necessary modules:

In [12]:
import math
import time
import tdmclient.notebook
await tdmclient.notebook.start()            #cant run with aseba opened!


Variables can now be accessed directly from the Jupyter workspace. They can be written and read.

Variables have similar names to the Aseba, with '.' replaced by '_' (because dots have a different meaning in Python). For example, the variable of the center front sensor, 'prox.horizontal[2]' becomes 'prox_horizontal[2]' and the variable to set the speed of the left motor 'motor.left.target' becomes 'motor_left_target'. 

More info here: https://pypi.org/project/tdmclient/



In [4]:
d_sensor = prox_horizontal[2]
print(d_sensor)

4402


In [7]:
motor_left_target=0
motor_right_target=0

 It is also possible to  send a whole program to Thymio, written either in Aseba Python. Below, we'll give equivalent examples in both languages: change the robot's color depending on which button is touched. The first line %%run_aseba allows to send the content of the cell (aseba code) to the robot and run it.

In [8]:
%%run_aseba
onevent buttons
    if button.center==1 then
        motor.left.target=0
        motor.right.target=0
    elseif button.forward==1 then
        motor.left.target=100
        motor.right.target=100
    elseif button.backward==1 then
        motor.left.target=-100
        motor.right.target=-100
    elseif button.left==1 then
        motor.left.target=50
        motor.right.target=100
    elseif button.right==1 then
        motor.left.target=100
        motor.right.target=50
    end

The equivalent in Python. Events have to be defined as functions, and decorated with @onevent. If Thymio variables are used inside the event, they should be declared as global (by default they would be considered local variables). This time, %%run_python is used, allowing to send the content of the cell (python code) to the robot and run it. 

**Note**: Some features (variables, events, etc.) are not available on the simulator

In [9]:
%%run_python
@onevent
def buttons():
    global button_center, button_forward, button_backward, button_left, button_right, leds_top
    if button_center==1: 
        leds_top=[32,0,0]
    elif button_forward==1: 
        leds_top=[0,32,0]
    elif button_backward==1: 
        leds_top=[0,0,32]
    elif button_left==1: 
        leds_top=[32,32,0]
    elif button_right==1: 
        leds_top=[0,32,32]

Changing the LEDs next to horizontal proximity sensors using a function. The first argument is the front left LED, then clockwise. Note that in python, the aseba native functions are prefixed with `nf_`. Events are emitted using the syntax `emit("name",[arg1, arg2])`

In [11]:
%%run_python
nf_leds_prox_h(0,32,0,32,0,32,0,32) 
v=[0,0]
nf_math_rand(v)
emit("ola",v[0],v[1])

## Communicating between the computer and the robot

It is possible to run programs on the PC, reading and updating the variables on the robot. 

In [ ]:
while True:
    dh=get_var("prox_horizontal")
    sl=int(dh[0][5]/10) #dh is a tuple, 5,6 are sensors on the back of the robot 
    sr=int(dh[0][6]/10)
    set_var(motor_right_target=sr,motor_left_target=sl)
    time.sleep(0.1)

Or use the decorator `@tdmclient.notebook.sync_var` on a function to sync all variables before and after the function 

In [ ]:
@tdmclient.notebook.sync_var
def andar(left,right):
    global motor_right_target, motor_left_target
    motor_right_target=right
    motor_left_target=left

@tdmclient.notebook.sync_var
def read_prox():
    global prox_horizontal
    #print(f'Prox: {prox_horizontal[1]},{prox_horizontal[3]}')
    return prox_horizontal[1],prox_horizontal[3]

while True:
    l,r=read_prox()
    andar(l//20,r//20)
    time.sleep(0.1)


It is also possible to send and receive events between the robot and the computer, etc.
For example, load the following program onto the robot:

In [ ]:
%%run_python
@onevent
def buttons():
    global button_center, button_forward, button_backward, button_left, button_right, leds_top, acc
    if button_center==1: 
        leds_top=[32,0,0]        
    elif button_forward==1: 
        leds_top=[0,32,0]
        emit("front",acc[0],acc[1],acc[2])
    elif button_backward==1: 
        emit("back",acc[0],acc[1],acc[2])
        leds_top=[0,0,32]        
    elif button_left==1: 
        leds_top=[32,32,0]
    elif button_right==1: 
        leds_top=[0,32,32]

In [ ]:
%%run_python
@onevent
def buttons():
    global button_center, button_forward, button_backward, button_left, button_right, leds_top, acc
    if button_forward==1:         
        emit("front",acc[0],acc[1],acc[2])
    elif button_backward==1: 
        emit("back",acc[0],acc[1],acc[2])

And run the following on the PC:
The decorator `@tdmclient.notebook.sync_var` means that the variables will be updated before and after the function below. `tdmclient.notebook.process_events(...)` will wait for events from the robot(s) and process them through function `on_event_data()`

In [ ]:
leds=[32,0,32,0,32,0,32,0]

@tdmclient.notebook.sync_var
def lights_on(l_diff):        
    global leds_circle
    for i,l in enumerate(leds):        
        leds[i]=min(max(l+l_diff,0),32)
    leds_circle = leds

def on_event_data(node, event_name):
    data=get_event_data(event_name)
    print(data)
    if event_name=="front":
        lights_on(1)
    if event_name=="back":
        lights_on(-1)
    clear_event_data(event_name)
        

tdmclient.notebook.process_events(all_nodes=True, on_event_data=on_event_data)


# List of Local Events

| event | description | frequency (Hz) | result |
| :- | :- | :- | :- |
| button_backward | back arrow was pressed or released | upon action | button.backward |
| button_left | left arrow was pressed or released | upon action | button.left |
| button_center | central button was pressed or released | upon action | button.center |
|button_forward|front arrow was pressed or released|upon action|button.forward|
|button_right|right arrow was pressed or released|upon action|button.right|
|buttons|button values have been probed|50|buttons.backward, buttons.left, buttons.center, buttons.forward, buttons.right|
|prox|proximity sensors were read|10|prox.horizontal[0-7], prox.ground.ambiant[0-1], prox.ground.reflected[0-1] and prox.ground.delta[0-1] |
|prox_comm|value received from IR sensors|upon value reception|prox.comm.rx|
|tap|a shock was detected|upon shock|acc[0-2] |
|acc|the accelerometer was read|16|acc[0-2] |
|mic|ambient sound intensity was above threshold|when condition is true|mic.intensity|
|sound_finished|a sound started by aseba has finished playing by itself|when sound finishes|
|temperature|temperature was read|1|temperature|
|rc5|the infrared remote-control receiver got a signal|upon signal reception|rc5.address and rc5.command|
|motor|PID is executed 	100|motor.left/right.speed, motor.left/right.pwm |
|timer0|when timer 0 period expires|user-defined||
|timer1|when timer 1 period expires|user-defined||